In [1]:
import streamlit as st
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt
import joblib

In [2]:
# 加载模型
model = joblib.load("C:\\Users\\lenovo\\毕设-20212133050\\XGBoost.pkl")

In [ ]:
# 页面标题
st.title('Alzheimer\'s Disease Prediction')

feature_names = ['Age', 'Sex', 'GLU', 'CREA', 'DBIL', 'IBIL', 'UA', 'AST', 'ALT', 'UREA', 'TBIL', 'BUN/Scr']

# 创建表单
with st.form("data_form"):
    age = st.number_input('年龄', min_value=0, max_value=120, value=50, step=1)
    sex = st.selectbox("性别", options=['男', '女'])
    sex_value = 1 if sex == '男' else 0

    glu = st.number_input('GLU（血糖，mg/dL）', min_value=0.0, max_value=200.0, 
                          value=80.0, step=0.1, 
                          help="正常范围：空腹血糖 70 - 99 mg/dL；餐后2小时 < 140 mg/dL")
    if sex == '男':
        crea = st.number_input('CREA（肌酐，mg/dL）', min_value=0.0, max_value=2.0, 
                               value=1.0, step=0.01, 
                               help="正常范围：0.74 - 1.35 mg/dL")
    else:
        crea = st.number_input('CREA（肌酐，mg/dL）', min_value=0.0, max_value=2.0, 
                               value=0.8, step=0.01, 
                               help="正常范围：0.59 - 1.04 mg/dL")
    dbil = st.number_input('DBIL（直接胆红素，mg/dL）', min_value=0.0, max_value=1.0, 
                           value=0.2, step=0.01, 
                           help="正常范围：0.1 - 0.3 mg/dL")
    ibil = st.number_input('IBIL（间接胆红素，mg/dL）', min_value=0.0, max_value=2.0, 
                           value=0.5, step=0.01, 
                           help="正常范围：0.2 - 0.8 mg/dL")
    if sex == '男':
        ua = st.number_input('UA（尿酸，mg/dL）', min_value=0.0, max_value=10.0, 
                             value=5.0, step=0.1, 
                             help="正常范围：3.4 - 7.0 mg/dL")
    else:
        ua = st.number_input('UA（尿酸，mg/dL）', min_value=0.0, max_value=10.0, 
                             value=4.0, step=0.1, 
                             help="正常范围：2.4 - 6.0 mg/dL")
    ast = st.number_input('AST（天冬氨酸氨基转移酶，U/L）', min_value=0.0, max_value=100.0, 
                          value=25.0, step=0.1, 
                          help="正常范围：10 - 40 U/L")
    alt = st.number_input('ALT（丙氨酸氨基转移酶，U/L）', min_value=0.0, max_value=100.0, 
                          value=30.0, step=0.1, 
                          help="正常范围：7 - 56 U/L")
    urea = st.number_input('UREA（尿素，mg/dL）', min_value=0.0, max_value=30.0, 
                           value=15.0, step=0.1, 
                           help="正常范围：7 - 20 mg/dL")
    tbil = st.number_input('TBIL（总胆红素，mg/dL）', min_value=0.0, max_value=2.0, 
                           value=0.7, step=0.01, 
                           help="正常范围：0.1 - 1.2 mg/dL")
    bun_scr = st.number_input('BUN/Scr（血尿素氮与肌酐比率）', min_value=0.0, max_value=30.0, 
                              value=15.0, step=0.1, 
                              help="正常范围：10:1 到 20:1")

    submit_button = st.form_submit_button(label='提交')

if submit_button:
    try:
        feature_values = [age, sex_value, glu, crea, dbil, ibil, ua, ast, alt, urea, tbil, bun_scr]
        features = np.array([feature_values], dtype=np.float32)

        st.write('您输入的参数如下:')
        st.write(f'年龄: {age}')
        st.write(f'性别: {sex}')
        st.write(f'GLU（血糖）: {glu} mg/dL')
        st.write(f'CREA（肌酐）: {crea} mg/dL')
        st.write(f'DBIL（直接胆红素）: {dbil} mg/dL')
        st.write(f'IBIL（间接胆红素）: {ibil} mg/dL')
        st.write(f'UA（尿酸）: {ua} mg/dL')
        st.write(f'AST（天冬氨酸氨基转移酶）: {ast} U/L')
        st.write(f'ALT（丙氨酸氨基转移酶）: {alt} U/L')
        st.write(f'UREA（尿素）: {urea} mg/dL')
        st.write(f'TBIL（总胆红素）: {tbil} mg/dL')
        st.write(f'BUN/Scr（血尿素氮与肌酐比率）: {bun_scr}')

        # 检查输入值是否在正常范围内
        normal_range_checks = []
        if not (70 <= glu <= 99):
            normal_range_checks.append("GLU（血糖）值不在正常空腹血糖范围（70 - 99 mg/dL）内，请注意！")
        if sex == '男' and not (0.74 <= crea <= 1.35):
            normal_range_checks.append("CREA（肌酐）值不在男性正常范围（0.74 - 1.35 mg/dL）内，请注意！")
        elif sex == '女' and not (0.59 <= crea <= 1.04):
            normal_range_checks.append("CREA（肌酐）值不在女性正常范围（0.59 - 1.04 mg/dL）内，请注意！")
        if not (0.1 <= dbil <= 0.3):
            normal_range_checks.append("DBIL（直接胆红素）值不在正常范围（0.1 - 0.3 mg/dL）内，请注意！")
        if not (0.2 <= ibil <= 0.8):
            normal_range_checks.append("IBIL（间接胆红素）值不在正常范围（0.2 - 0.8 mg/dL）内，请注意！")
        if sex == '男' and not (3.4 <= ua <= 7.0):
            normal_range_checks.append("UA（尿酸）值不在男性正常范围（3.4 - 7.0 mg/dL）内，请注意！")
        elif sex == '女' and not (2.4 <= ua <= 6.0):
            normal_range_checks.append("UA（尿酸）值不在女性正常范围（2.4 - 6.0 mg/dL）内，请注意！")
        if not (10 <= ast <= 40):
            normal_range_checks.append("AST（天冬氨酸氨基转移酶）值不在正常范围（10 - 40 U/L）内，请注意！")
        if not (7 <= alt <= 56):
            normal_range_checks.append("ALT（丙氨酸氨基转移酶）值不在正常范围（7 - 56 U/L）内，请注意！")
        if not (7 <= urea <= 20):
            normal_range_checks.append("UREA（尿素）值不在正常范围（7 - 20 mg/dL）内，请注意！")
        if not (0.1 <= tbil <= 1.2):
            normal_range_checks.append("TBIL（总胆红素）值不在正常范围（0.1 - 1.2 mg/dL）内，请注意！")
        if not (10 <= bun_scr <= 20):
            normal_range_checks.append("BUN/Scr（血尿素氮与肌酐比率）值不在正常范围（10:1 到 20:1）内，请注意！")

        if normal_range_checks:
            st.warning("以下指标值不在正常范围内：")
            for check in normal_range_checks:
                st.write(check)

        # 模型预测
        predicted_class = model.predict(features)[0]
        predicted_proba = model.predict_proba(features)[0]

        st.write(f"**预测类别:** {predicted_class}")
        st.write(f"**预测概率:** {predicted_proba[predicted_class]:.2f}%")

        # 生成建议
        probability = predicted_proba[predicted_class] * 100
        if predicted_class == 1:
            advice = f"根据预测结果，您患阿尔茨海默病的概率为 {probability:.2f}%。建议您及时就医，进行进一步的专业检查和诊断。同时，保持健康的生活方式，如均衡饮食、适度运动、充足睡眠等。"
        else:
            advice = f"根据预测结果，您目前患阿尔茨海默病的概率较低，为 {probability:.2f}%。但仍建议您保持定期体检，维持健康的生活习惯，以预防疾病的发生。"
        st.write(advice)

        # 生成 SHAP 图
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(pd.DataFrame([feature_values], columns=feature_names))
        fig = shap.force_plot(explainer.expected_value, shap_values[0], pd.DataFrame([feature_values], columns=feature_names), matplotlib=True)
        plt.savefig("shap_force_plot.png", bbox_inches='tight', dpi=120)
        plt.close()  # 关闭图像
        st.image("shap_force_plot.png")
    except Exception as e:
        st.error(f"预测过程中出现错误: {str(e)}")

In [ ]:

# 页面标题
st.title('Alzheimer\'s Disease Prediction')

feature_names = ['Age', 'Sex', 'GLU', 'CREA', 'DBIL', 'IBIL', 'UA', 'AST', 'ALT', 'UREA', 'TBIL', 'BUN/Scr']

# 创建表单
with st.form("data_form"):
    age = st.number_input('年龄', min_value=0, max_value=120, value=50, step=1)
    sex = st.selectbox("Sex (0=Female, 1=Male):", 
                       options=[0, 1], format_func=lambda x: 'Female (0)' if x == 0 else 'Male (1)')
    st.write("You selected:", "Female" if sex == 0 else "Male")
    glu = st.number_input('GLU', min_value=0.0, max_value=100.0, value=50.0, step=0.1)
    crea = st.number_input('CREA', min_value=0.0, max_value=100.0, value=50.0, step=0.1)
    dbil = st.number_input('DBIL', min_value=0.0, max_value=100.0, value=50.0, step=0.1)
    ibil = st.number_input('IBIL', min_value=0.0, max_value=100.0, value=50.0, step=0.1)
    ua = st.number_input('UA', min_value=0.0, max_value=100.0, value=50.0, step=0.1)
    ast = st.number_input('AST', min_value=0.0, max_value=100.0, value=50.0, step=0.1)
    alt = st.number_input('ALT', min_value=0.0, max_value=100.0, value=50.0, step=0.1)
    urea = st.number_input('UREA', min_value=0.0, max_value=100.0, value=50.0, step=0.1)
    tbil = st.number_input('TBIL', min_value=0.0, max_value=100.0, value=50.0, step=0.1)
    bun_scr = st.number_input('BUN/Scr', min_value=0.0, max_value=100.0, value=50.0, step=0.1)
    
    submit_button = st.form_submit_button(label='提交')

if submit_button:
    feature_values = [age, sex, glu, crea, dbil, ibil, ua, ast, alt, urea, tbil, bun_scr]
    features = np.array([feature_values], dtype=np.float32)
    
    st.write('您输入的参数如下:')
    st.write(f'年龄: {age}')
    st.write(f'性别: {"Female" if sex == 0 else "Male"}')
    st.write(f'GLU: {glu}')
    # 其他参数显示...
    
    predicted_class = model.predict(features)[0]
    predicted_proba = model.predict_proba(features)[0]
    
    st.write(f"**预测类别:** {predicted_class}")
    st.write(f"**预测概率:** {predicted_proba[predicted_class]:.2f}%")
    
    # 生成建议
    probability = predicted_proba[predicted_class] * 100
    if predicted_class == 1:
        advice = "..."  # 你的建议内容
    else:
        advice = "..."  # 你的建议内容
    st.write(advice)
    
    # 生成SHAP图
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(pd.DataFrame([feature_values], columns=feature_names))
    shap.force_plot(explainer.expected_value, shap_values[0], pd.DataFrame([feature_values], columns=feature_names), matplotlib=True)
    plt.savefig("shap_force_plot.png", bbox_inches='tight', dpi=120)
    plt.close()  # 关闭图像
    st.image("shap_force_plot.png")